In [2]:
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [4]:
model=KeyedVectors.load_word2vec_format("wiki.pa.vec")

In [5]:
vocab=[]
for word in model.vocab:
    vocab.append(word)

In [16]:
stopwords = ['ਹੋਣ', 'ਆਪਣੇ', 'ਕਰਕੇ', 'ਲਿਆ', 'ਹੋਇਆ', 'ਤੱਕ', 'ਲੈ', 'ਰਹੇ', 'ਦੱਸਿਆ', 'ਪੀ', 'ਕੌਰ', 'ਸਿੰਘ', 'ਗਿਆ',
             'ਦੀਆਂ', 'ਦਿੱਤਾ', 'ਜਿਸ', 'ਜਿਸ', 'ਕੀਤੀ', 'ਰਿਹਾ', 'ਕਿਹਾ', 'ਟੀ', 'ਰਹੀ', 'ਜਾ', 'ਹੋਏ', 'ਤਾਂ', 'ਹੋ', 'ਦੇ',
             'ਨੂੰ', 'ਤੋਂ', 'ਪਰ', 'ਇਹ', 'ਹੀ', 'ਵ', 'ਦਾ', 'ਨੇ', 'ਕਿ', 'ਤੇ', 'ਦੀ', 'ਦੇ', 'ਨੂੰ', 'ਇਕ', 'ਬਾਰੇ',
             'ਉਪਰੋਕਤ', 'ਭਰ', 'ਬਾਅਦ', 'ਬਾਅਦ', 'ਦੁਬਾਰਾ', 'ਵਿਰੁੱਧ', 'ਸਭ', 'ਲਗਭਗ', 'ਇਕੱਲੀ', 'ਨਾਲ', 'ਪਹਿਲਾਂ ਹੀ',
             'ਵੀ', 'ਹਾਲਾਂ', 'ਹਮੇਸ਼ਾ', 'ਵਿਚ', 'ਵਿਚਕਾਰ', 'ਅਮਨਗਸਟ', 'ਮਾਤਰਾ', 'ਇਕ', 'ਅਤੇ', 'ਇਕ ਹੋਰ', 'ਕੋਈ',
             'ਕਿਸੇ','ਵੀ','ਤਰ੍ਹਾਂ', 'ਕਿਸੇ ਵੀ', 'ਕੁਝ ਵੀ', 'ਕਿਸੇ ਵੀ ਜਗ੍ਹਾ', 'ਕਿਤੇ ਵੀ', 'ਹਨ', 'ਆਲੇ', 'ਬੈਕ', 'ਬਣ',
             'ਬਣ ਗਿਆ', 'ਬਣਨ', 'ਬਣ', 'ਬਾਹਰੀ', 'ਇਲਾਵਾ', 'ਵਿਚਕਾਰ', 'ਪਰੇ', 'ਬਿੱਲ', 'ਦੋਨੋ', 'ਤਲ', 'ਪਰ', 'ਬਣ', 'ਕੇ',
             'ਕਾਲ', 'ਕੈਨ', 'ਨਹੀਂ', 'ਕੈਨਟ', 'ਕੋ', 'ਕੰਪਿਊਟਰ', 'ਕੋਨ', 'ਸੀ', 'ਬਰਦਾਸ਼ਤ', 'ਰੋਏ', 'ਡੀ', 'ਵਰਣਨ',
             'ਵਿਸਥਾਰ', 'ਕੀਤਾ', 'ਕਰਨਾ', 'ਕੀਤਾ', 'ਹੇਠਾਂ', 'ਹਰ', 'ਜਿਵੇਂ', 'ਅੱਠ', 'ਜਾਂ', 'ਹੋਰ', 'ਹੋਰ', 'ਖਾਲੀ',
             'ਕਾਫ਼ੀ', 'ਆਦਿ', 'ਵੀ', 'ਕਦੇ', 'ਹਰ', 'ਹਰ ਕੋਈ', 'ਹਰ ਚੀਜ਼', 'ਹਰ ਜਗ੍ਹਾ', 'ਸਿਵਾਏ', 'ਕੁਝ', 'ਪੰਦਰਾਂ',
             'ਪੰਜਾਹ', 'ਭਰਨ', 'ਲੱਭੋ', 'ਪਹਿਲਾ', 'ਪੰਜ', 'ਲਈ', 'ਪੁਰਾਣਾ', 'ਪਹਿਲਾਂ', 'ਚਾਲੀ', 'ਮਿਲਿਆ', 'ਚਾਰ', 'ਤੋ',
             'ਫਰੰਟ', 'ਪੂਰਾ', 'ਅੱਗੇ', 'ਪ੍ਰਾਪਤ', 'ਦੇਣਾ', 'ਜਾਓ', 'ਹੈ', 'ਹੈ', 'ਹੈ ਨਾ', 'ਹੈ', 'ਉਹ', 'ਇਸ', 'ਉਸ',
             'ਇੱਥੇ', 'ਅੱਗੇ', 'ਏਸ ਕੇ', 'ਏਥੇ', 'ਹੁਣ', 'ਉਸ', 'ਆਪ', 'ਉਸ', 'ਖ਼ੁਦ', 'ਉਸ', 'ਕਿਸ', 'ਪਰ', 'ਸੌ', 'ਮੈਂ',
             'ਅਰਥਾਤ', 'ਜੇ', 'ਇਨ', 'ਅਸਲ', 'ਵਿਆਜ', 'ਇਨ', 'ਐੱਸ', 'ਇਹ', 'ਆਖ', 'ਬਾਅਦ', 'ਬਾਅਦ', 'ਘੱਟ', 'ਘੱਟ', 'ਲਿਡ',
             'ਬਣਾਇਆ', 'ਬਹੁਤ ਸਾਰੇ', 'ਮੇ', 'ਮੈਂ', 'ਇਸ ਦੌਰਾਨ', 'ਸ਼ਾਇਦ', 'ਮਿੱਲ', 'ਮੇਰਾ', 'ਹੋਰ', 'ਹੋਰ', 'ਸਭ',
             'ਜ਼ਿਆਦਾਤਰ', 'ਮੂਵ', 'ਬਹੁਤ', 'ਜ਼ਰੂਰ', 'ਮੇਰਾ', 'ਮੈਂ', 'ਨਾਮ', 'ਅਰਥਾਤ', 'ਨਾ', 'ਕਦੇ ਨਹੀਂ', 'ਫਿਰ', 'ਅਗਲਾ',
             'ਨੌਂ', 'ਨਹੀਂ', 'ਕੋਈ ਨਹੀਂ', 'ਕੋਈ ਨਹੀਂ', 'ਨਾ', 'ਕੁਝ ਨਹੀਂ', 'ਹੁਣ', 'ਕਿਤੇ ਨਹੀਂ', 'ਦਾ', 'ਬੰਦ', 'ਅਕਸਰ',
             'ਇਕ ਵਾਰ', 'ਇਕ', 'ਸਿਰਫ', 'ਉੱਤੇ', 'ਜਾਂ', 'ਹੋਰ', 'ਹੋਰ', 'ਹੋਰ', 'ਸਾਡਾ', 'ਸਾਡਾ', 'ਆਪ', 'ਬਾਹਰ', 'ਓਵਰ',
             'ਖੁਦ', 'ਹਿੱਸਾ', 'ਉਹੀ', ' ਵੇਖੋ', 'ਜਾਪ ', 'ਵੇਖਣਾ', 'ਲੱਗਦਾ', 'ਗੰਭੀਰ', 'ਕਈ', 'ਉਹ', 'ਕਰਨਾ ਚਾਹੀਦਾ ਹੈ',
             'ਸ਼ੋਅ', 'ਸਾਈਡ', 'ਏ', 'ਈਰਿਵਾਰ', 'ਛੇ', 'ਸੱਠ', 'ਉਸ', 'ਉਹ', 'ਉਨ੍ਹਾਂ', 'ਉਨ੍ਹਾਂ', 'ਆਪ', 'ਤਦ', 'ਤਦ ਤੋਂ',
             'ਉੱਥੇ', 'ਉਸ ਤੋਂ ਬਾਅਦ', 'ਇਸ ਤਰ੍ਹਾਂ', 'ਇਸ ਲਈ', 'ਉਸ ਵਿਚ', 'ਇਸ ਤੋਂ ਬਾਅਦ', 'ਇਹ', 'ਉਹ', 'ਮੋਟੀ', 'ਪਤਲੇ',
             'ਤੀਸਰਾ', 'ਇਹ', 'ਉਹ', 'ਭਾਵੇਂ', 'ਤਿੰਨ', 'ਤਿੰਨ', 'ਪੂਰੇ', 'ਥਰੂ', 'ਇਸ ਤਰ੍ਹਾਂ', 'ਇਕੋ', 'ਬਹੁਤ', 'ਚੋਟੀ',
             'ਵੱਲ', 'ਵੱਲ', 'ਬਾਰਾਂ', 'ਵੀਹ', 'ਦੋ', 'ਅਣ', 'ਅਧੀਨ', 'ਤਦ', 'ਅਪ', 'ਉੱਤੇ', 'ਸਾਨੂੰ', 'ਬਹੁਤ', 'ਰਾਹ', 'ਸੀ',
             'ਅਸੀਂ', 'ਚੰਗੀ', 'ਸਨ', 'ਕੀ', 'ਜੋ','ਵੀ', 'ਕਦ', 'ਕਦੋਂ', 'ਜਦੋਂ ਵੀ', 'ਕਿੱਥੇ', 'ਬਾਅਦ', 'ਜਦੋਂ', 'ਜਿਸ ਵਿਚ ',
             'ਜਿਸ ਵਿਚ', 'ਜਿਧਰ', 'ਕਿਤੇ ਵੀ', 'ਕੀ', 'ਕਿਹੜਾ', 'ਕਦ', 'ਕਿਥੇ', 'ਕੌਣ']


stopwords_split=[i.split()[0] for i in stopwords]

In [6]:
len(vocab)

56850

In [11]:
vocab[45:55]

['ਸੀ', 'ਉਹ', 'ਜੋ', 'ਹੀ', '।', 'ਦੀਆਂ', 'ਭਾਰਤ', 'for', '}}}', 'data']

In [75]:
model.vectors[0].shape

(300,)

In [73]:
search='ਇੱਕ'
search2='ਮੁੰਬਈ'
search3='ਡੱਡੂ'
search4='ਵਾਹਿਗੁਰੂ'

In [74]:
model.similar_by_word(search3)

[('ਡੱਡੂਆਂ', 0.7903650999069214),
 ('ਲੂੰਬੜੀ', 0.7376791834831238),
 ('ਲੂੰਬੜ', 0.7309291362762451),
 ('ਬਿੱਛੂ', 0.7178257703781128),
 ('ਕੱਛੂਕੁੰਮਾ', 0.7090239524841309),
 ('ਛਿੱਲੜ', 0.7007120847702026),
 ('ਲੱਡੂ', 0.699851393699646),
 ('ਆਲ੍ਹਣੇ', 0.6943454742431641),
 ('ਆਲ੍ਹਣਾ', 0.6878112554550171),
 ('ਜਲਥਲੀ', 0.6864452958106995)]

In [18]:
data=pd.read_csv("punjabi_newsss.csv")


In [19]:
data.head()

,Article_id,Article_title,Article_Content
0,1,NaN,ਮੁੰਬਈ ਬਿਊਰੋ ਬਾਲੀਵੁੱਡ ਅਕਸ਼ੈ ਕੁਮਾਰ ਨੇ ਮੁੰਬਈ ਹਾਲ ਹ...
1,2,ਹੁਣ ਮਾਈਕ੍ਰੋਸਾਫਟ ਵੀ ਬਦਲੇਗਾ ਪਿਸਤੌਲ ਇਮੋਜੀ,ਜਲੰਧਰ ਗੂਗਲ ਤੇ ਫੇਸਬੁੱਕ ਵੱਲੋਂ ਪਿਸਤੌਲ ਇਮੋਜੀ ਨੂੰ ਵ...
2,3,NaN,ਮੁੰਬਈ ਬਿਊਰੋ ਬਾਲੀਵੁੱਡ ਅਭਿਨੇਤਰੀ ਸੋਹਾ ਅਲੀ ਖਾਨ ਦੀ ...
3,4,ਟਰੈਕਟਰ ਤੇ ਮੋਟਰਸਾਈਕਲ ਦੀ ਟੱਕਰ ਜ਼ਖਮੀ,ਜੈਤੋ ਜਿੰਦਲ – ਬੀਤੇ ਦਿਨ ਰੋੜੀਕਪੂਰਾ ਤੋਂ ਕਾਸਮਭੱਟੀ ਰ...
4,5,ਸਾਲ ਕਰੋੜ ਰੁਪਏ ਪੂੰਜੀ ਵਪਾਰ ਕਰੇਗੀ ਮਾਰੂਤੀ ਸੁਜ਼ੂਕੀ,ਨਵੀਂ ਦਿੱਲੀ ਕਾਰ ਬਣਾਉਣ ਵਾਲੀ ਦੇਸ਼ ਦੀ ਤੋਂ ਵੱਡੀ ਕੰਪਨ...


In [20]:
# data.Article_Content
corpus=[]
for article in data.Article_Content:
    if type(article) == str:
#         if len(article)<10:
#             pass
        corpus.append(re.sub(" \d+",'',article))

In [33]:
tfid=TfidfVectorizer(stop_words=stopwords_split)

In [34]:
tfid_mat=tfid.fit_transform(corpus)

In [35]:
tfid_mat

<2969x5518 sparse matrix of type '<class 'numpy.float64'>'
	with 155013 stored elements in Compressed Sparse Row format>

In [76]:
fasttext_vocab_matrix=[]
for word in tfid.vocabulary_:
    if word in model.vocab:
        fasttext_vocab_matrix.append(model.get_vector(word))
    else:
        fasttext_vocab_matrix.append(np.zeros((300)))
        
fasttext_vocab_matrix=np.array(fasttext_vocab_matrix)

In [77]:
fasttext_vocab_matrix.shape

(5518, 300)

In [78]:
tfid_mat.shape,fasttext_vocab_matrix.shape

((2969, 5518), (5518, 300))

In [79]:
vec=np.dot(tfid_mat.todense(),fasttext_vocab_matrix)

In [81]:
print(vec.shape)

(2969, 300)


In [82]:
cos_sims=cosine_similarity(vec)

In [83]:
cos_sims.shape

(2969, 2969)

In [85]:
# cosine_similarity of one article with itself is 1 ,it will be first recommendation for that article so we remove 
# all entries from similarity matrix (cos_sims) with value gearter than 0.99
cos_sims[cos_sims>0.99]=0      

In [86]:
cos_sim_sorted=cos_sims.argsort()

In [87]:
def top_5_similar(story_num,sims_sorted):
    indexes=sims_sorted[story_num][-5:]
#     print(indexes)
    similar_articles=[data.Article_Content[i] for i in indexes[::-1]]
    return similar_articles
  

In [88]:
corpus[1936]

'ਤਹਿਰਾਨ ਦੱਖਣੀ ਈਰਾਨ ਦੇ ਪਰਬਤੀ ਖੇਤਰ ਵਿਚ ਅੱਜ ਤੀਬਰਤਾ ਦਾ ਭੂਚਾਲ ਜਿਸ ਵਿਚ ਘੱਟ ਤੋਂ ਘੱਟ ਲੋਕ ਜ਼ਖਮੀ ਹੋ ਗਏ। ਨਾਲ ਬਿਜਲੀ ਸਪਲਾਈ ਅਤੇ ਸੰਚਾਰ ਵਿਵਸਥਾ ਪ੍ਰਭਾਵਿਤ ਹੋਈ ਹੈ। ਸਰਕਾਰੀ ਮੀਡੀਆ ਦੀ ਵਿਚ ਕਿਹਾ ਗਿਆ ਹੈ ਕਿ ਭੂਚਾਲ ਤਹਿਰਾਨ ਤੋਂ ਕਰੀਬ ਕਿਲੋਮੀਟਰ ਦੱਖਣ ਵਿਚ ਸਿਸਖਤ ਸ਼ਹਿਰ ਵਿਚ ਆਇਆ। ਦਹਿਸ਼ਤ ਦੀ ਵਜ੍ਹਾ ਨਾਲ ਲੋਕ ਸੜਕਾਂ ਤੇ ਗਏ। ਤੁਹਾਨੂੰ ਦੱਸ ਕਿ ਸ਼ਹਿਰ ਦੀ ਅਬਾਦੀ ਹੈ। ਟੈਲੀਵਿਜ਼ਨ ਫੁਟੇਜ ਵਿਚ ਦਿਖਾਇਆ ਗਿਆ ਕਿ ਇਮਾਰਤਾਂ ਦੀਆਂ ਕੰਧਾਂ ਵਿਚ ਦਰਾੜਾ ਗਈਆਂ ਹਨ। ਸਰਕਾਰੀ ਗੱਲਬਾਤ ਕਮੇਟੀ ਮੁਤਾਬਕ ਸੁਬਾਈ ਐਮਰਜੈਂਸੀ ਸੇਵਾਵਾਂ ਦੇ ਮੁਖੀ ਜਲਾਲ ਪੈਰਾਨਫਰਡ ਨੇ ਕਿਹਾ ਕਿ ਲੋਕਾਂ ਨੂੰ ਮੈਡੀਕਲ ਕੇਂਦਰਾਂ ਵਿਚ ਭੇਜਿਆ ਗਿਆ ਹੈ ਅਤੇ ਹੋਰ ਲੋਕਾਂ ਦਾ ਇਲਾਜ ਕੀਤਾ ਗਿਆ ਹੈ। ਈਰਾਨ ਵਿਚ ਭੂਚਾਲ ਆਉਂਦੇ ਰਹਿੰਦੇ ਹਨ। ਪਿਛਲੇ ਸਾਲ ਨਵੰਬਰ ਵਿਚ ਪੱਛਮੀ ਈਰਾਨ ਵਿਚ ਤੀਬਰਤਾ ਦਾ ਭੂਚਾਲ ਸੀ। ਵਿਚ ਤੋਂ ਜ਼ਿਆਦਾ ਲੋਕਾਂ ਦੀ ਮੌਤ ਹੋ ਸੀ। ਇਸੇ ਤਰ੍ਹਾਂ ਵਿਚ ਤੀਬਰਤਾ ਦੇ ਭੂਚਾਲ ਵਿਚ ਇਤਿਹਾਸਕ ਸ਼ਹਿਰ ਬਾਮ ਤਬਾਹ ਹੋ ਗਿਆ ਸੀ। ਉਦੋਂ ਲੋਕਾਂ ਦੀ ਮੌਤ ਹੋ ਸੀ।'

In [89]:
top_5_similar(1936,cos_sim_sorted)

['ਸਿਡਨੀ ਦੁਨੀਆ ਤੋਂ ਲੰਬੇ ਸਮੇਂ ਤੱਕ ਜਿਊਂਦੀ ਰਹਿਣ ਵਾਲੀ ਮੱਕੜੀ ਦੀ 43 ਸਾਲ ਦੀ ਵਿਚ ਮੌਤ ਹੋ ਗਈ। ਆਸਟ੍ਰੇਲੀਆ ਵਿਚ ਜਨਸੰਖਿਆ ਅਧਿਐਨ ਦੌਰਾਨ ਲੰਬੇ ਸਮੇਂ ਤੋਂ ਮੱਕੜੀ ਤੇ ਰੱਖੀ ਜਾ ਰਹੀ ਸੀ। ਖੋਜਕਰਤਾਵਾਂ ਨੇ ਮੱਕੜੀ ਦਾ ਨਾਂ ਨੰਬਰ 16 ਰੱਖਿਆ ਹੋਇਆ ਸੀ। ਖੋਜਕਰਤਾਵਾਂ ਨੇ ਦੱਸਿਆ ਕਿ ਮੱਕੜੀ ਦੀ ਮੌਤ ਬੁੱਢੇ ਹੋਣ ਕਾਰਨ ਨਹੀਂ ਸਗੋਂ ਕਿ ਨੂੰ ਕੀੜੇ ਨੇ ਆਪਣਾ ਸ਼ਿਕਾਰ ਬਣਾ ਜਾਣ ਕਾਰਨ ਹੋਈ ਹੈ। ਮੱਕੜੀ ਦੀ ਵਜ੍ਹਾ ਤੋਂ ਵਿਗਿਆਨੀਆਂ ਨੂੰ ਸਮੁੱਚੇ ਆਸਟ੍ਰੇਲੀਆ ਵਿਚ ਪਾਈਆਂ ਜਾਣ ਵਾਲੀਆਂ ਮੱਕੜੀਆਂ ਦੇ ਵਿਵਹਾਰ ਨੂੰ ਵਿਚ ਮਿਲੀ। ਤੋਂ ਪਹਿਲਾਂ ਤੋਂ ਲੰਬੇ ਸਮੇਂ ਤੱਕ ਜ਼ਿੰਦਾ ਰਹਿਣ ਦਾ ਰਿਕਾਰਡ ਮੈਕਸੀਕੋ ਦੇ ਤਰਾਨਤੁਲਾ ਮੱਕੜੀ ਦੇ ਨਾਂ ਸੀ ਜੋ ਕਿ 28 ਸਾਲ ਤੱਕ ਜਿਊਂਦੀ ਰਹੀ। ਪੈਸਫਿਕ ਕੰਜ਼ਰਵੇਸ਼ਨ ਬਾਇਓਲਾਜੀ ਵਿਚ ਪ੍ਰਕਾਸ਼ਤ ਅਧਿਐਨ ਮੁਤਾਬਕ ਟਰੈਪਡੁਰ ਮੱਕੜੀ ਨੇ ਤਰਾਨਤੁਲਾ ਮੱਕੜੀ ਦੇ 28 ਸਾਲ ਤੱਕ ਜਿਊਂਦੇ ਰਹਿਣ ਦੇ ਰਿਕਾਰਡ ਨੂੰ ਤੋੜ ਦਿੱਤਾ ਹੈ। ਟਰੈਪਡੁਰ ਮੱਕੜੀਆਂ ਦਾ ਜੀਵਨਕਾਲ 5 ਤੋਂ 20 ਸਾਲ ਤੱਕ ਦਾ ਹੁੰਦਾ ਹੈ। ਮੱਕੜੀਆਂ ਮਨੁੱਖ ਖਤਰਾ ਤਾਂ ਨਹੀਂ ਇਨ੍ਹਾਂ ਦੇ ਕੱਟਣ ਨਾਲ ਹੁੰਦਾ ਹੈ ਅਤੇ ਸੋਜ ਹੋ ਜਾਂਦੀ ਹੈ।',
 'ਮੁੰਬਈ ਬਿਊਰੋ ਸ਼ੁੱਕਰਵਾਰ 27 ਅਪ੍ਰੈਲ ਤੋਂ ਸੋਸ਼ਲ ਮੀਡੀਆ ਤੇ ਵਾਇਰਲ ਹੋ ਰਹੀ ਹੈ ਕਿ ਮਸ਼ਹੂਰ ਅਦਾਕਾਰਾ ਮੁਮਤਾਜ਼ ਦਾ ਦਿਹਾਂਤ ਹੋ ਗਿਆ ਹੈ ਅਫਵਾਹ ਤੇ ਦੀ ਬੇਟੀ ਤਾਨਿਆ ਮਾਧਵਾਨੀ ਨੇ ਵਿਰਾਮ ਲਾ ਦਿੱਤਾ ਹੈ। ਤਾਨਿਆ ਨੇ ਆਪਣੇ ਇੰਸਟਾਗ੍ਰਾਮ ਤੇ ਵੀਡੀ

In [89]:
##################  projection part ####################

In [91]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector

In [94]:
file=open("metadata.tsv",'w')
for news in corpus:
    file.write(news + "\n")
file.close()

In [129]:
with tf.Session() as sess:
    embedding_variable=tf.Variable(vec,name="embedding")
    sess.run(tf.global_variables_initializer())
    # create a TensorFlow summary writer
    summary_writer = tf.summary.FileWriter('log', sess.graph)
    config = projector.ProjectorConfig()
    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = 'embedding'
    embedding_conf.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(summary_writer, config)

    # save the model
    saver = tf.train.Saver()
    saver.save(sess, 'log/model.ckpt')